# Merging SOEP with weather data

This notebook merges the weatherdata downloaded from `meteostat` with the SOEP household panel data. Since this merging can only be done based on timestamps and geographic locations of each household the resulting quality of the merged dataset highly depends on the granularity of the SOEP dataset. For now the merging has to be performed on a **NUTS 1 level** (Bundesebene) as more granular data can only be accessed at the secure data center in Berlin.

The merging of the data follows the specifications of the paper *Hue et al.*

In [1]:
import pandas as pd
import numpy as np
import random
from weather.src.helper import read_nuts_weather_data

## Reading the weather data

The weatherdata stems from the website `meteostat.com`. The contents are downloaded and prepared in the subfolder `./weather`. Also some helper functions can be found in the folder `./weather/src/helper.py`. Those functions convert data from individual stations into SOEP compatible format. For example the `read_nuts_weather_data` takes all the stations within the same NUTS compartment and takes the mean of the station data.

One thing that needs to be addressed is that the soep data uses different geographic naming convetions than meteostat.
In the following is the conversion chart for the NUTS level 1:
| Meteostat | SOEP | Name |
|-----|----|------------------------|
| DEF | 1  | Schleswig-Holstein     |
| DE6 | 2  | Hamburg                |
| DE9 | 3  | Niedersachsen          |
| DE5 | 4  | Bremen                 |
| DEA | 5  | Nordrhein-Westfalen    |
| DE7 | 6  | Hessen                 |
| DEB | 7  | Rheinland-Pfalz        |
| DE1 | 8  | Baden-Wuerttemberg     |
| DE2 | 9  | Bayern                 |
| DEC | 10 | Saarland               |
| DE3 | 11 | Berlin                 |
| DE4 | 12 | Brandenburg            |
| DE8 | 13 | Mecklenburg-Vorpommern |
| DED | 14 | Sachsen                |
| DEE | 15 | Sachsen-Anhalt         |
| DEG | 16 | Thuringen              |

In [2]:
# reading the weather data for NUTS 1 area codes
weather = read_nuts_weather_data('./weather/prod/weatherdata/nuts1', bar=False)
weather['time'] = pd.to_datetime(weather['time'])

# rename variables
chart = {
    "DE6" : 2, "DEF" : 1, "DE9" : 3, "DE5" : 4, "DEA" : 5, "DE7" : 6, "DEB" : 7, "DE1" : 8, "DE2" : 9, 
    "DEC" : 10, "DE3" : 11, "DE4" : 12, "DE8" : 13, "DED" : 14, "DEE" : 15, "DEG" : 16
}
weather["NUTS_CODE"].replace(chart, inplace=True)
# rename the column s.t. it matches the soep variable
weather.rename(columns={'NUTS_CODE':'bula_h'}, inplace=True)

# drop unusefull columns and set new index
weather.drop(["wdir", "wpgt", "elevation", "tmin"], axis=1, inplace=True)
weather.head()

,time,tavg,tmax,prcp,snow,wspd,pres,tsun,bula_h
0,1985-01-01,-1.455556,-0.368889,7.133333,40.000000,19.326667,1011.508333,5.777778,8
1,1985-01-02,-3.862222,-0.400000,5.442222,113.333333,14.513333,1006.208333,0.222222,8
2,1985-01-03,-6.426667,-4.477778,4.691111,170.666667,11.366667,1011.950000,44.222222,8
3,1985-01-04,-11.495556,-5.075556,1.668889,229.555556,12.793333,1008.608333,202.888889,8
4,1985-01-05,-13.100000,-10.142222,0.720000,239.555556,5.526667,1012.908333,102.666667,8


## Computing climate variables

This particular step follows the data preparation steps taken in *Hue et al.*. In the following these steps.
1. Bin the data into 5 degrees intervals (first and last interval are open starting at -5 degrees going up to 35 degrees)
2. Calculate for each month how many days fall within each bin.
3. For other weather related variables (e.g. pressure/hours of sun) the monthly average is taken.

In the following you find all these steps.

In [3]:
# binning the data into 5 degree intervals
bins = [-float('inf')]+[x for x in range(0, 31, 5)]+[float('inf')]
labels = [x for x in range(len(bins)-1)]
weather['tavgbin'] = pd.cut(weather['tavg'], bins, labels=labels)
weather.head()

,time,tavg,tmax,prcp,snow,wspd,pres,tsun,bula_h,tavgbin
0,1985-01-01,-1.455556,-0.368889,7.133333,40.000000,19.326667,1011.508333,5.777778,8,0
1,1985-01-02,-3.862222,-0.400000,5.442222,113.333333,14.513333,1006.208333,0.222222,8,0
2,1985-01-03,-6.426667,-4.477778,4.691111,170.666667,11.366667,1011.950000,44.222222,8,0
3,1985-01-04,-11.495556,-5.075556,1.668889,229.555556,12.793333,1008.608333,202.888889,8,0
4,1985-01-05,-13.100000,-10.142222,0.720000,239.555556,5.526667,1012.908333,102.666667,8,0


In [4]:
# calculate for each month how many days fall within each month
dummies1 = pd.get_dummies(weather[['tavgbin']]) # explodes the interal column
weather = pd.concat([weather, dummies1], axis=1)
weather.set_index('time', inplace=True)
weather.head()

,tavg,tmax,prcp,snow,wspd,pres,tsun,bula_h,tavgbin,tavgbin_0,tavgbin_1,tavgbin_2,tavgbin_3,tavgbin_4,tavgbin_5,tavgbin_6,tavgbin_7
time,,,,,,,,,,,,,,,,,
1985-01-01,-1.455556,-0.368889,7.133333,40.000000,19.326667,1011.508333,5.777778,8,0,1,0,0,0,0,0,0,0
1985-01-02,-3.862222,-0.400000,5.442222,113.333333,14.513333,1006.208333,0.222222,8,0,1,0,0,0,0,0,0,0
1985-01-03,-6.426667,-4.477778,4.691111,170.666667,11.366667,1011.950000,44.222222,8,0,1,0,0,0,0,0,0,0
1985-01-04,-11.495556,-5.075556,1.668889,229.555556,12.793333,1008.608333,202.888889,8,0,1,0,0,0,0,0,0,0
1985-01-05,-13.100000,-10.142222,0.720000,239.555556,5.526667,1012.908333,102.666667,8,0,1,0,0,0,0,0,0,0


In [5]:
# applying the counter for each month and calculating the average for the other clim. vars.
weather:pd.DataFrame

# define for which column what aggregation function is used
aggs = {f"tavgbin_{x}":np.sum for x in range(0, 8)}
aggs['prcp'], aggs['tsun'], aggs['wspd'], aggs['pres'], aggs['snow'] = (np.mean for i in range(5))
aggs['tavg'] = [np.std, np.mean]

# aggregate by month
weather = weather.groupby('bula_h').rolling('30D', min_periods=30).agg(aggs).dropna()

weather.rename({'tavg':'tavg_std'}, axis=1)
weather.head()

tavgbin_0 tavgbin_1 tavgbin_2 tavgbin_3 tavgbin_4 tavgbin_5  \
                        sum       sum       sum       sum       sum       sum   
bula_h time                                                                     
1      1985-01-30      24.0       6.0       0.0       0.0       0.0       0.0   
       1985-01-31      23.0       7.0       0.0       0.0       0.0       0.0   
       1985-02-01      22.0       7.0       1.0       0.0       0.0       0.0   
       1985-02-02      21.0       8.0       1.0       0.0       0.0       0.0   
       1985-02-03      21.0       8.0       1.0       0.0       0.0       0.0   

                  tavgbin_6 tavgbin_7      prcp    tsun       wspd  \
                        sum       sum      mean    mean       mean   
bula_h time                                                          
1      1985-01-30       0.0       0.0  1.435769  110.78  20.032381   
       1985-01-31       0.0       0.0  1.489084  113.60  20.450476   
       1985-02-01       0.0       0.0  1.535751  112.14  20.959048   
       1985-02-02       0.0       0.0  1.499799  104.66  21.385714   
       1985-02-03       0.0       0.0  1.491941  112.58  20.877143   

                          pres       snow      tavg            
                          mean       mean       std      mean  
bula_h time                                                    
1      1985-01-30  1012.164667  96.371429  4.145051 -4.595746  
       1985-01-31  1011.961333  95.226190  4.438277 -4.409365  
       1985-02-01  1011.640667  93.700000  4.766347 -4.153365  
       1985-02-02  1011.484667  91.988095  4.978070 -3.826032  
       1985-02-03  1012.098667  89.561905  4.974892 -3.659444

In [8]:
# after aggregation we need to reset the multiindex
weather.columns = weather.columns.map(
    lambda x: x[0] + '-' + x[1] if x[0] == 'tavg' else x[0]
)
weather

tavgbin_0  tavgbin_1  tavgbin_2  tavgbin_3  tavgbin_4  \
bula_h time                                                                
1      1985-01-30       24.0        6.0        0.0        0.0        0.0   
       1985-01-31       23.0        7.0        0.0        0.0        0.0   
       1985-02-01       22.0        7.0        1.0        0.0        0.0   
       1985-02-02       21.0        8.0        1.0        0.0        0.0   
       1985-02-03       21.0        8.0        1.0        0.0        0.0   
...                      ...        ...        ...        ...        ...   
16     2022-11-24        2.0        5.0       14.0        6.0        3.0   
       2022-11-25        2.0        6.0       14.0        5.0        3.0   
       2022-11-26        2.0        7.0       14.0        4.0        3.0   
       2022-11-27        2.0        8.0       14.0        4.0        2.0   
       2022-11-28        2.0        9.0       14.0        4.0        1.0   

                   tavgbin_5  tavgbin_6  tavgbin_7      prcp        tsun  \
bula_h time                                                                
1      1985-01-30        0.0        0.0        0.0  1.435769  110.780000   
       1985-01-31        0.0        0.0        0.0  1.489084  113.600000   
       1985-02-01        0.0        0.0        0.0  1.535751  112.140000   
       1985-02-02        0.0        0.0        0.0  1.499799  104.660000   
       1985-02-03        0.0        0.0        0.0  1.491941  112.580000   
...                      ...        ...        ...       ...         ...   
16     2022-11-24        0.0        0.0        0.0  1.002024  207.029167   
       2022-11-25        0.0        0.0        0.0  1.032738  203.479167   
       2022-11-26        0.0        0.0        0.0  1.042143  194.520833   
       2022-11-27        0.0        0.0        0.0  1.041488  191.820833   
       2022-11-28        0.0        0.0        0.0  1.073124  188.066667   

                        wspd         pres       snow  tavg-std  tavg-mean  
bula_h time                                                                
1      1985-01-30  20.032381  1012.164667  96.371429  4.145051  -4.595746  
       1985-01-31  20.450476  1011.961333  95.226190  4.438277  -4.409365  
       1985-02-01  20.959048  1011.640667  93.700000  4.766347  -4.153365  
       1985-02-02  21.385714  1011.484667  91.988095  4.978070  -3.826032  
       1985-02-03  20.877143  1012.098667  89.561905  4.974892  -3.659444  
...                      ...          ...        ...       ...        ...  
16     2022-11-24  11.642429  1015.454915   3.511905  4.888420   7.674350  
       2022-11-25  11.678644  1015.425650   3.536596  4.876271   7.431469  
       2022-11-26  11.678249  1015.577458   3.536596  4.786651   7.142373  
       2022-11-27  11.594068  1015.707571   3.536596  4.543719   6.676497  
       2022-11-28  11.654531  1015.488339   3.536596  4.290585   6.284610  

[220842 rows x 15 columns]

## Read soep data

Final step is to merge the mered SOEP data in `soeplong.ipynb` which the climate data. The key on which is merged is the NUTS 1 level and the timestamp. As mentioned the NUTS 1 level variable is included in the SOEP dataset for each household.

In [9]:
# read soep data
soep = pd.read_csv('./prod/soeplong.csv')

# merge with weather df
soep['time'] = pd.to_datetime(soep['time'])
soep.set_index(['bula_h', 'time'], inplace=True)
# join and drop nan values
soep = soep.join(weather)

# free up some memory
del weather 

# save dataset
soep.to_csv('./prod/data.csv')

In [10]:
soep

Unnamed: 0         pid     index        hid    year  \
bula_h time                                                              
1.0    1985-03-02        5285     18801.0   12580.0     1880.0  1985.0   
       1985-03-02        5321     18802.0   12581.0     1880.0  1985.0   
       1985-03-02        5551     19301.0   12588.0     1937.0  1985.0   
       1985-03-02        5561     19302.0   12589.0     1937.0  1985.0   
       1985-03-04        5405     19001.0   12582.0     1902.0  1985.0   
...                       ...         ...       ...        ...     ...   
16.0   2020-12-16      706046  37701301.0  736587.0  3770133.0  2020.0   
       2020-12-16      706060  37702501.0  736589.0  3770257.0  2020.0   
       2020-12-17      713148  38719001.0  737635.0  3871908.0  2020.0   
       2020-12-22      722167  55214001.0  741757.0  5521400.0  2020.0   
NaN    1997-03-07       17251     62503.0  159857.0    88366.0  1997.0   

                   plb0021  plh0171  plh0175  plh0182  partner  ...  \
bula_h time                                                     ...   
1.0    1985-03-02      2.0      7.0      8.0      7.0      1.0  ...   
       1985-03-02      2.0      3.0      8.0      6.0      1.0  ...   
       1985-03-02      2.0      8.0      7.0      7.0      2.0  ...   
       1985-03-02      2.0      9.0      7.0      7.0      2.0  ...   
       1985-03-04      1.0      9.0      5.0      8.0      2.0  ...   
...                    ...      ...      ...      ...      ...  ...   
16.0   2020-12-16      2.0     10.0     -5.0      6.0      0.0  ...   
       2020-12-16      1.0      8.0     -5.0      6.0      0.0  ...   
       2020-12-17      2.0      5.0     -5.0     10.0      1.0  ...   
       2020-12-22     -5.0      7.0     -5.0      9.0      0.0  ...   
NaN    1997-03-07      2.0      8.0      7.0      8.0      0.0  ...   

                   tavgbin_5  tavgbin_6  tavgbin_7      prcp        tsun  \
bula_h time                                                                
1.0    1985-03-02        0.0        0.0        0.0  0.489066  153.910909   
       1985-03-02        0.0        0.0        0.0  0.489066  153.910909   
       1985-03-02        0.0        0.0        0.0  0.489066  153.910909   
       1985-03-02        0.0        0.0        0.0  0.489066  153.910909   
       1985-03-04        0.0        0.0        0.0  0.538590  160.025455   
...                      ...        ...        ...       ...         ...   
16.0   2020-12-16        0.0        0.0        0.0  0.397627  102.323333   
       2020-12-16        0.0        0.0        0.0  0.397627  102.323333   
       2020-12-17        0.0        0.0        0.0  0.379096  104.560000   
       2020-12-22        0.0        0.0        0.0  0.675028  110.060000   
NaN    1997-03-07        NaN        NaN        NaN       NaN         NaN   

                        wspd         pres      snow  tavg-std  tavg-mean  
bula_h time                                                               
1.0    1985-03-02  16.067679  1022.710889  6.516484  4.536406  -2.601762  
       1985-03-02  16.067679  1022.710889  6.516484  4.536406  -2.601762  
       1985-03-02  16.067679  1022.710889  6.516484  4.536406  -2.601762  
       1985-03-02  16.067679  1022.710889  6.516484  4.536406  -2.601762  
       1985-03-04  14.883333  1023.257556  6.516484  4.466091  -2.645143  
...                      ...          ...       ...       ...        ...  
16.0   2020-12-16  11.976222  1016.154158  3.547619  2.643619   2.149792  
       2020-12-16  11.976222  1016.154158  3.547619  2.643619   2.149792  
       2020-12-17  11.884667  1016.039396  3.547619  2.386897   1.999896  
       2020-12-22  11.685111  1014.838681  3.547619  2.306511   1.961562  
NaN    1997-03-07        NaN          NaN       NaN       NaN        NaN  

[722382 rows x 36 columns]